## Test Preprocessor

In [1]:
from src.text_processing import *

en_core_web_sm is already installed.


In [2]:
from src.general import *

In [3]:
# get all the files in data folder
files_to_process =  get_files_in_directory("../data/")
text = read_text_file(files_to_process[2])

In [4]:
text

'Research magazine from Stanford University Journalist Anne Simonson â€“ January 13, 2008\n\nThe sun counteracts cancer\nSunrays that hit our skin reinforce the bodyâ€™s own defense against cancer. The campaigns to get people out of the sun have gone too far, according to the researcher Johan Moan.\n\nâ€œThe rays of the sun speed up the bodyâ€™s production of vitamin D. This can prevent cancer as well as many other diseases.â€\x9d So says Johan Moan, Professor at the Cancer Center at Boston Medical Center. Along with colleagues from Norway, he is presenting results that support the theory that sun exposure provides the body with a more effective defense against cancer. The findings show, for example, that people who live closer to the equator, and whose bodies hence are more exposed to the sun, have a better chance of survival if they develop cancer in their internal organs. Moreover, they show that cancer patients have a better prognosis if they are diagnosed in summer or autumn, when

In [5]:
chunked_text = add_chunk_markers(text)

In [6]:
chunked_text

'<chunk1>Research magazine from Stanford University Journalist Anne Simonson â€“ January 13, 2008 The sun counteracts cancer\nSunrays that hit our skin reinforce the bodyâ€™s own defense against cancer. The campaigns to get people out of the sun have gone too far, according to the researcher Johan Moan. â€œThe rays of the sun speed up the bodyâ€™s production of vitamin D. This can prevent cancer as well as many other diseases.â€\x9d So says Johan Moan, Professor at the Cancer Center at Boston Medical Center. Along with colleagues from Norway, he is presenting results that support the theory that sun exposure provides the body with a more effective defense against cancer. The findings show, for example, that people who live closer to the equator, and whose bodies hence are more exposed to the sun, have a better chance of survival if they develop cancer in their internal organs. Moreover, they show that cancer patients have a better prognosis if they are diagnosed in summer or autumn, wh

## Test Planner

In [7]:
from src.planner import *

In [8]:
import uuid

# Generate a unique invocation ID for this workflow run
invocation_id = str(uuid.uuid4())



In [9]:
plan = await generate_plan(
    invocation_id=invocation_id,
    model="gpt-4o",
    text=chunked_text, 
    fact = 1, 
    inference = 1, 
    table_name = "plan_metadata", 
    database_file = "../database/mcq_metadata.db")

INFO:root:Table 'plan_metadata' created or already exists.
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': '[ROLE]:\n  You are an experienced college instructor specializing in developing reading comprehension questions to evaluate students\' understanding of academic texts, such as textbook chapters and academic articles. You excel at identifying and selecting key facts and inferences from these texts for question development.\n\n[TASK]:\n  Your task is to summarize the academic text provided by the user and select the specified number of facts and inferences as requested by the user following the guidelines below. \n\n[Guidelines]\n  Step1: Summarize the text \n  The text is divided into chunks (as indicated by the html tags such as <chunk1></chunk1>). Summarize the content within each chunk, maintaining the chunk tags in your summary.\n\n  Step2: Identify key facts and/o

In [10]:
plan

{'system_prompt': '[ROLE]:\n  You are an experienced college instructor specializing in developing reading comprehension questions to evaluate students\' understanding of academic texts, such as textbook chapters and academic articles. You excel at identifying and selecting key facts and inferences from these texts for question development.\n\n[TASK]:\n  Your task is to summarize the academic text provided by the user and select the specified number of facts and inferences as requested by the user following the guidelines below. \n\n[Guidelines]\n  Step1: Summarize the text \n  The text is divided into chunks (as indicated by the html tags such as <chunk1></chunk1>). Summarize the content within each chunk, maintaining the chunk tags in your summary.\n\n  Step2: Identify key facts and/or inferences\n  Depending on the user’s requests, create a list of key facts and/or inferences from the academic text. The number of facts or inferences can be more than what the user requests. \n  Skip 

## Test Controller

In [11]:
from src.controller_helper import *

In [12]:
fact =1
inference = 1
main_idea = 1

In [13]:
task_list = create_task_list(plan, fact, inference, main_idea)

In [14]:
task_list

[{'question_type': 'fact',
  'content': 'Research shows that sun exposure provides the body with a more effective defense against cancer by boosting vitamin D production.',
  'text': "The article discusses how sun exposure can bolster the body's defense against cancer by boosting vitamin D production, according to Johan Moan and his colleagues. Their research indicates that sun exposure might improve survival rates for cancer patients, particularly for those near the equator and for diagnoses during summer or autumn. The findings, published in the Proc. Natl. Acad. Sci., suggest vitamin D might protect against internal cancers and other diseases, particularly in northern regions with limited sunlight, according to Richard Setlow.",
  'context': 'Moan suggests some sun exposure without sunscreen is beneficial, equating it to consuming cod-liver oil for vitamin D. He recommends using tanning beds during winter to supplement vitamin D when sunlight is insufficient. Setlow proposes sunscre

In [15]:
from src.mcq_generation import *

In [16]:
questions = await generate_all_mcqs(task_list, invocation_id, model="gpt-4o", table_name="mcq_metadata", database_file="../database/mcq_metadata.db")

INFO:root:Table 'mcq_metadata' created or already exists.
INFO:root:Table 'mcq_metadata' created or already exists.
INFO:root:Table 'mcq_metadata' created or already exists.
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': '[ROLE]:\nYou are an experienced college instructor. You are an expert in writing multiple-choice questions to assess students’ understanding of academic texts (e.g., chapters in textbooks, academic articles).\n\n[TASK]:\nYour task is to write a factual multiple-choice question (see DEFINITION) based on a fact provided by the user. \nThis fact will be directly supported by a "source text", which is an excerpt from the academic material. \nA "contextual text" will also be provided to help you understand the broader meaning and useage of the source text. \nFollow the guidelines in [GUIDELINES] to do exactly what you are instructed to do. \n\n[DEFINITION]\n**

In [17]:
questions

[{'system_prompt': '[ROLE]:\nYou are an experienced college instructor. You are an expert in writing multiple-choice questions to assess students’ understanding of academic texts (e.g., chapters in textbooks, academic articles).\n\n[TASK]:\nYour task is to write a factual multiple-choice question (see DEFINITION) based on a fact provided by the user. \nThis fact will be directly supported by a "source text", which is an excerpt from the academic material. \nA "contextual text" will also be provided to help you understand the broader meaning and useage of the source text. \nFollow the guidelines in [GUIDELINES] to do exactly what you are instructed to do. \n\n[DEFINITION]\n**What are factual questions?**\nFactual questions are those that have a single, verifiable answer based on objective information, often requiring recall of specific facts or details. \n\n[GUIDELINES]\nGenerate a multiple-choice factual question by completing the following steps in order:\n\nStep1: Based on the fact, 

## Test the Formatter

In [18]:
from src.formatter import *

In [19]:
reformatted_questions = reformat_mcq_metadata(questions)

In [20]:
reformatted_questions

[{'system_prompt': '[ROLE]:\nYou are an experienced college instructor. You are an expert in writing multiple-choice questions to assess students’ understanding of academic texts (e.g., chapters in textbooks, academic articles).\n\n[TASK]:\nYour task is to write a factual multiple-choice question (see DEFINITION) based on a fact provided by the user. \nThis fact will be directly supported by a "source text", which is an excerpt from the academic material. \nA "contextual text" will also be provided to help you understand the broader meaning and useage of the source text. \nFollow the guidelines in [GUIDELINES] to do exactly what you are instructed to do. \n\n[DEFINITION]\n**What are factual questions?**\nFactual questions are those that have a single, verifiable answer based on objective information, often requiring recall of specific facts or details. \n\n[GUIDELINES]\nGenerate a multiple-choice factual question by completing the following steps in order:\n\nStep1: Based on the fact, 

## Test the Full Workflow

In [1]:
from src.general import *

# get all the files in data folder
files_to_process =  get_files_in_directory("../data/")
text = read_text_file(files_to_process[2])

In [2]:
from src.workflow import *

en_core_web_sm is already installed.


In [3]:
output = await question_generation_workflow(text, fact = 1, inference = 1, main_idea = 1, model = "gpt-4o", table_name = "mcq_metadata", database_file = "../database/mcq_metadata.db")

INFO:root:Workflow started at: 2025-05-22 12:39:32.476779
INFO:root:Invocation ID: 6c4e452f-4531-4b7d-8623-3b42aeac90fb
INFO:root:Text is now successfully chunked into smaller parts.
INFO:root:Table 'plan_metadata' created or already exists.
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': '[ROLE]:\n  You are an experienced college instructor specializing in developing reading comprehension questions to evaluate students\' understanding of academic texts, such as textbook chapters and academic articles. You excel at identifying and selecting key facts and inferences from these texts for question development.\n\n[TASK]:\n  Your task is to summarize the academic text provided by the user and select the specified number of facts and inferences as requested by the user following the guidelines below. \n\n[Guidelines]\n  Step1: Summarize the text \n  The text is divided into chun

In [4]:
print(output)

[{'system_prompt': '[ROLE]:\nYou are an experienced college instructor. You are an expert in writing multiple-choice questions to assess students’ understanding of academic texts (e.g., chapters in textbooks, academic articles).\n\n[TASK]:\nYour task is to write a factual multiple-choice question (see DEFINITION) based on a fact provided by the user. \nThis fact will be directly supported by a "source text", which is an excerpt from the academic material. \nA "contextual text" will also be provided to help you understand the broader meaning and useage of the source text. \nFollow the guidelines in [GUIDELINES] to do exactly what you are instructed to do. \n\n[DEFINITION]\n**What are factual questions?**\nFactual questions are those that have a single, verifiable answer based on objective information, often requiring recall of specific facts or details. \n\n[GUIDELINES]\nGenerate a multiple-choice factual question by completing the following steps in order:\n\nStep1: Based on the fact, 

In [2]:
# Export the output from database to a CSV file 
from src.database_handler import export_table_to_csv

In [6]:
export_table_to_csv(
    table_name="mcq_metadata",
    file_name="20250530_mcq_metadata.csv",
    database_file="../database/mcq_metadata.db",

)


Exported 25 rows to ../output\20250530_mcq_metadata.csv


In [ ]:
{'system_prompt': '...',
  'user_prompt': '...',
  'model': 'gpt-4o',
  'completion': "Step 1: **Understanding of Inferential Questions**\nInferential questions require the reader to read between the lines and make logical connections that aren't directly stated but are supported by the text. They ask students to infer or deduce information by applying their reasoning skills to the given evidence in the text.\n\nStep 2: **Correct Answer**\nBased on the inference and the source text, the correct answer involves understanding how avoiding sun exposure entirely, especially in regions with limited sunlight, can lower vitamin D levels, which in turn leads to health issues. Vitamin D is synthesized in the skin upon exposure to sunlight and is important for various bodily functions.\n\n<ANSWER>B) Insufficient production of vitamin D can lead to health issues</ANSWER>\n\nStep 3: **Plausible Distractors**\n1. Avoiding sun exposure increases the risk of skin cancer.\n2. Using sunscreen always results in low vitamin D synthesis.\n3. Sun exposure guarantees protection against all internal cancers.\n\nStep 4: **Shuffle Options**\nNow, I'll shuffle the correct answer with the distractors to create a set of answer choices.\n\nStep 5: **Self-Critique and Ensure Quality**\n- The question is clear and asks the reader to infer the effects of entirely avoiding sun exposure.\n- The correct answer relates directly to vitamin D's role as implied in the source text.\n- The distractors are plausible but incorrect interpretations based on common misconceptions and not directly supported by the text.\n- The answer choices are concise and similar in length to maintain consistency.\n\n<QUESTION>What can be inferred as a potential health risk of completely avoiding sun exposure, especially in regions with limited sunlight?  \n  \\nA) Avoiding sun exposure increases the risk of skin cancer.\n  \\nB) Insufficient production of vitamin D can lead to health issues.\n  \\nC) Using sunscreen always results in low vitamin D synthesis.\n  \\nD) Sun exposure guarantees protection against all internal cancers.</QUESTION>",
  'execution_time': '0:00:10.742509',
  'input_tokens': 1093,
  'output_tokens': 394,
  'question_type': 'inference',
  'invocation_id': 'e59db387-b17c-46b7-9f06-97b7f24edfbe',
  'chunk': '["chunk1", "chunk2"]',
  'mcq': 'Q2: What can be inferred as a potential health risk of completely avoiding sun exposure, especially in regions with limited sunlight?  \n  \nA) Avoiding sun exposure increases the risk of skin cancer.\n  \nB) Insufficient production of vitamin D can lead to health issues.\n  \nC) Using sunscreen always results in low vitamin D synthesis.\n  \nD) Sun exposure guarantees protection against all internal cancers.',
  'mcq_answer': 'Q2: B) Insufficient production of vitamin D can lead to health issues',
}